In [ ]:
"""
Runs forward passes and stores hidden states, topk-selections, and intermediate activations
"""
None

In [ ]:
"""
Imports
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.loss.loss_utils import ForCausalLMLoss # Cross-entropy loss that handles label shifting
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
from termcolor import colored
import importlib
import os

from utils.memory import check_memory, clear_all_cuda_memory
from utils.store_topk import convert_topk_to_df
from utils.store_outputs import convert_outputs_to_df
from utils import pretrained_models

import pickle

main_device = 'cuda:0'
seed = 1234
clear_all_cuda_memory()
check_memory()

## Load base model

In [ ]:
"""
Load the base tokenizer/model

Architectures supported currently:
- OlMoE architecture, includes OLMoE-1B-7B-0125-Instruct (1B/7B)
- Qwen2MoE architecture, inclues Qwen1.5-MoE-A2.7B-Chat (2.7B/14.3B), Qwen2-57B-A14B (14B/57B)
- Deepseek v2 architecture, includes Deepseek-v2-Lite (2.4B/15.7B), Deepseek-v2 (21B/236B)
- Deepseek v3 architecture, includes Deepseek-v3 (37B/671B), Deepseek-R1 (37B/671B), Moonlight-16B-A3B (3B/16B)
"""
selected_model_index = 1

def get_model(index):
    model = [
        ('allenai/OLMoE-1B-7B-0125-Instruct', 'olmoe', 'olmoe'),
        ('Qwen/Qwen1.5-MoE-A2.7B-Chat', 'qwen1.5moe', 'qwen2moe'),
        ('deepseek-ai/DeepSeek-V2-Lite', 'dsv2', 'dsv2'),
        ('moonshotai/Moonlight-16B-A3B', 'moonlight', 'dsv3')
    ][index]

    return model[0], model[1], model[2]

model_id, model_prefix, model_architecture = get_model(selected_model_index)
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = False, add_bos_token = False, padding_side = 'left', trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, trust_remote_code = True).cuda().eval()

In [ ]:
"""
Load reverse-engineered forward pass functions that return topk expert IDs and weights
"""
model_module = importlib.import_module(f"utils.pretrained_models.{model_architecture}")
run_model_return_topk = getattr(model_module, f"run_{model_architecture}_return_topk")

def test_custom_forward_pass(model, pad_token_id):
    inputs = tokenizer(['Hi! I am a dog and I like to bark', 'Vegetables are good for'], return_tensors = 'pt', padding = 'max_length', truncation = True, max_length = 512).to(model.device)
    original_results = model(**inputs)
    custom_results = run_model_return_topk(model, inputs['input_ids'], inputs['attention_mask'], return_hidden_states = True)
    assert torch.equal(original_results.logits, custom_results['logits']), 'Error in custom forward'
    assert len(custom_results['all_topk_experts']) == len(custom_results['all_topk_weights']), 'Length of topk IDs and weights not equal'
    print(f"Length of topk: {len(custom_results['all_topk_experts'])}")
    print(f"Topk size: {custom_results['all_topk_experts'][0].shape}")
    print(f"First token topk IDs: {custom_results['all_topk_experts'][0][1,]}")
    print(f"First token topk weights: {custom_results['all_topk_weights'][0][1,]}")
    loss = ForCausalLMLoss(custom_results['logits'], torch.where(inputs['input_ids'] == pad_token_id, torch.tensor(-100), inputs['input_ids']), model.config.vocab_size).detach().cpu().item()
    print(f"LM loss: {loss}")
    print(f"Hidden states layers (pre-mlp | post-layer): {len(custom_results['all_pre_mlp_hidden_states'])} | {len(custom_results['all_hidden_states'])}")
    print(f"Hidden state size (pre-mlp | post-layer): {(custom_results['all_pre_mlp_hidden_states'][0].shape)} | {(custom_results['all_hidden_states'][0].shape)}")
    print(f"Expert outputs : {(custom_results['all_expert_outputs'][0].shape)}")

test_custom_forward_pass(model, tokenizer.pad_token_id)

## Get dataset

In [ ]:
"""
Load dataset - C4 mix (en/zh/es)
"""
def load_raw_ds():
   
    ds_en = load_dataset('allenai/c4', 'en', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 50_000)
    ds_zh = load_dataset('allenai/c4', 'zh', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 50_000)
    ds_es = load_dataset('allenai/c4', 'es', split = 'validation', streaming = True).shuffle(seed = 123, buffer_size = 50_000)
    
    def get_data(ds, n_samples, data_source):
        raw_data = []
        ds_iter = iter(ds)
        for _ in range(0, n_samples):
            sample = next(ds_iter, None)
            if sample is None:
                break
            raw_data.append({'text': sample['text'], 'source': data_source})
        
        return raw_data
    
    combined_ds = get_data(ds_en, 1200, 'en') + get_data(ds_zh, 300, 'zh') + get_data(ds_es, 300, 'es')
    
    return combined_ds

raw_data = load_raw_ds()

In [ ]:
""" 
Load dataset into a dataloader. The dataloader returns the original tokens - this is important for BPE tokenizers as otherwise it's difficult to reconstruct the correct string later!
"""
from torch.utils.data import Dataset, DataLoader

class ReconstructableTextDataset(Dataset):

    def __init__(self, text_dataset, tokenizer, max_length):
        """
        Creates a dataset object that also returns a B x N list of the original tokens in the same position as the input ids.

        Params:
            @text_dataset: A list of B samples of text dataset. Each sample is a dict with keys `text` and `source`.
            @tokenizer: A HF tokenizer object.
        """
        texts = [x['text'] for x in text_dataset]
        sources = [x['source'] for x in text_dataset]
        tokenized = tokenizer(texts, add_special_tokens = False, max_length = max_length, padding = 'max_length', truncation = True, return_offsets_mapping = True, return_tensors = 'pt')

        self.input_ids = tokenized['input_ids']
        self.sources = sources
        self.attention_mask = tokenized['attention_mask']
        self.offset_mapping = tokenized['offset_mapping']
        self.original_tokens = self.get_original_tokens(texts)

    def get_original_tokens(self, texts):
        """
        Return the original tokens associated with each B x N position. This is important for reconstructing the original text when BPE tokenizers are used.
        
        Params:
            @input_ids: A B x N tensor of input ids.
            @offset_mapping: A B x N x 2 tensor of offset mappings. Get from `tokenizer(..., return_offsets_mapping = True)`.

        Returns:
            A list of length B, each with length N, containing the corresponding original tokens corresponding to the token ID at the same position of input_ids.
        """
        all_token_substrings = []
        for i in range(0, self.input_ids.shape[0]):
            token_substrings = []
            for j in range(self.input_ids.shape[1]): 
                start_char, end_char = self.offset_mapping[i][j].tolist()
                if start_char == 0 and end_char == 0: # When pads, offset_mapping might be [0, 0], so let's store an empty string for those positions.
                    token_substrings.append("")
                else:
                    original_substring = texts[i][start_char:end_char]
                    token_substrings.append(original_substring)
            
            all_token_substrings.append(token_substrings)

        return all_token_substrings

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], 'original_tokens': self.original_tokens[idx], 'sources': self.sources[idx]}
    
def collate_fn(batch):
    """
    Custom collate function; necessary to return original_tokens in the correct shape 
    """
    input_ids = torch.stack([b['input_ids'] for b in batch], dim = 0)
    attention_mask = torch.stack([b['attention_mask'] for b in batch], dim = 0)        
    original_tokens = [b['original_tokens'] for b in batch]
    sources = [b['sources'] for b in batch]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'original_tokens': original_tokens, 'sources': sources}

test_dl = DataLoader(
    ReconstructableTextDataset(raw_data, tokenizer, max_length = 1024),
    batch_size = 16,
    shuffle = False,
    collate_fn = collate_fn
)

## Get expert selections + export

In [ ]:
""" 
Run forward passes + export data
"""

@torch.no_grad()
def run_and_export_topk(model, dl: ReconstructableTextDataset, act_layers_to_keep: list[int], topk_layers_to_keep = list[int], topk_to_keep: int = 0, max_batches: None | int = None):
    """
    Run forward passes on given model and store the intermediate hidden layers as well as topks

    Params:
        @model: The model to run forward passes on. Should return a dict with keys `logits`, `all_topk_experts`, `all_topk_weights`, and
          `all_pre_mlp_hidden_states`.
        @dl: The dataloader which returns `input_Ids`, `attention_mask`, and `original_tokens`.
        @act_layers_to_keep: A list of layer indices (0-indexed) for which to filter `all_pre_mlp_hidden_states` (see returned object description).
        @topk_layers_to_keep: A list of layer indices (0-indexed) for which to return topk-indices (see returned object description).
        @topk_to_keep: How many of the topk-indices for which to return in `all_expert_outputs` (see returned object description).
        @max_batches: The max number of batches to run.

    Returns:
        A dict with keys:
        - `sample_df`: A sample (token)-level dataframe with corresponding input token ID, output token ID, and input token text (removes masked tokens)
        - `topk_df`: A sample (token) x layer_ix x topk_ix level dataframe that gives the expert ID selected at each sample-layer-topk (removes masked_tokens)
        - `all_pre_mlp_hidden_states`: A tensor of size n_samples x act_layers_to_keep x D return the hidden state for each retained layers. Each 
            n_sample corresponds to a row of sample_df.
        - `all_expert_outputs` A tensor of size n_samples x topk_layers_to_keep x topk_to_keep x D to return the MLP output for all retained topk indices for each each retained layer.
    """
    b_count = 0
    all_pre_mlp_hidden_states = []
    all_expert_outputs = []
    sample_dfs = []
    topk_dfs = []

    for batch_ix, batch in tqdm(enumerate(dl), total = len(dl)):

        input_ids = batch['input_ids'].to(main_device)
        attention_mask = batch['attention_mask'].to(main_device)
        original_tokens = batch['original_tokens']
        sources = batch['sources']

        output = run_model_return_topk(model, input_ids, attention_mask, return_hidden_states = True)

        # Check no bugs by validating output/perplexity
        if batch_ix == 0:
            loss = ForCausalLMLoss(output['logits'], torch.where(input_ids == tokenizer.pad_token_id, torch.tensor(-100), input_ids), model.config.vocab_size).detach().cpu().item()
            for i in range(min(5, input_ids.size(0))):
                decoded_input = tokenizer.decode(input_ids[i, :], skip_special_tokens = True)
                next_token_id = torch.argmax(output['logits'][i, -1, :]).item()
                print('---------\n' + decoded_input + colored(tokenizer.decode([next_token_id], skip_special_tokens = False).replace('\n', '<lb>'), 'green'))
            print(f"PPL:", torch.exp(torch.tensor(loss)).item())
        
        original_tokens_df = pd.DataFrame(
            [(seq_i, tok_i, tok) for seq_i, tokens in enumerate(original_tokens) for tok_i, tok in enumerate(tokens)], 
            columns = ['sequence_ix', 'token_ix', 'token']
        )
                
        sources_df = pd.DataFrame(
            [(seq_i, seq_source) for seq_i, seq_source in enumerate(sources)], 
            columns = ['sequence_ix', 'source']
        )

        # Create sample (token) level dataframe
        sample_df =\
            convert_outputs_to_df(input_ids, attention_mask, output['logits'])\
            .merge(original_tokens_df, how = 'left', on = ['token_ix', 'sequence_ix'])\
            .merge(sources_df, how = 'left', on = ['sequence_ix'])\
            .assign(batch_ix = batch_ix)

        # Create topk x layer_ix x sample level dataframe
        topk_df =\
            convert_topk_to_df(input_ids, attention_mask, output['all_topk_experts'], output['all_topk_weights'])\
            .assign(batch_ix = batch_ix, weight = lambda df: df['weight'])\
            .drop(columns = 'token_id')#\
            # .pipe(lambda df: df[df['layer_ix'].isin(layers_to_keep)])
        
        sample_dfs.append(sample_df)
        topk_dfs.append(topk_df)

        # Store pre-MLP hidden states - the fwd pass as n_layers list as BN x D, collapse to BN x n_layers x D, with BN filtering out masked items
        valid_pos = torch.where(attention_mask.cpu().view(-1) == 1) # Valid (BN, ) positions
        all_pre_mlp_hidden_states.append(torch.stack(output['all_pre_mlp_hidden_states'], dim = 1)[valid_pos][:, act_layers_to_keep, :])
        all_expert_outputs.append(torch.stack(output['all_expert_outputs'], dim = 1)[valid_pos][:, topk_layers_to_keep, 0:topk_to_keep, :]) # This is BN x n_layers x topk x D - keep only top1 + top2

        b_count += 1
        if max_batches is not None and b_count >= max_batches:
            break

    return {
        'sample_df': pd.concat(sample_dfs),
        'topk_df': pd.concat(topk_dfs),
        'all_pre_mlp_hidden_states': torch.cat(all_pre_mlp_hidden_states, dim = 0), # n_samples x n_layers x D
        'all_expert_outputs': torch.cat(all_expert_outputs, dim = 0), # n_samples x n_layers x topk x D,
        'all_pre_mlp_hidden_states_layers': act_layers_to_keep,
        'all_expert_outputs_layers': topk_layers_to_keep 
    }

res = run_and_export_topk(model, test_dl, act_layers_to_keep = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 19, 21, 23], topk_layers_to_keep = [0, 3, 7, 11, 15], topk_to_keep = 2, max_batches = None)
# res = run_and_export_topk(model, test_dl, act_layers_to_keep = list(range(0, 16)), topk_layers_to_keep = [0, 3, 7, 11], topk_to_keep = 2, max_batches = None) # OlMoE

In [ ]:
"""
Save output
"""
output_dir = f'data/{model_prefix}'
os.makedirs(output_dir, exist_ok = True)

torch.save(res['all_pre_mlp_hidden_states'], f'{output_dir}/all-pre-mlp-hidden-states.pt')
del res['all_pre_mlp_hidden_states']
torch.save(res['all_expert_outputs'], f'{output_dir}/all-expert-outputs.pt')
del res['all_expert_outputs']
with open(f'{output_dir}/metadata.pkl', 'wb') as f:
    pickle.dump(
        {
            'sample_df': res['sample_df'], 
            'topk_df': res['topk_df'], 
            'all_pre_mlp_hidden_states_layers': res['all_pre_mlp_hidden_states_layers'],
            'all_expert_outputs_layers': res['all_expert_outputs_layers']
        },
        f
    )